In [240]:
# Biblioteki wejściowe

import pandas as pd
import numpy as np
import numpy_financial as npf
import posixpath as os
import pathlib
from pathlib import Path
import datetime
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython
import math

from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
#pd.options.display.min_colwidth=10
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

DATA = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/INWESTYCJE/LIBERTAS/DATA')
ASSETS = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/INWESTYCJE/LIBERTAS/ASSETS')
IMAGES = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/INWESTYCJE/LIBERTAS/IMAGES')

## Dane wejściowe

In [241]:
# Dane wejściowe modelu

t_remonty=pd.read_excel(DATA /'Remonty_Obligacje.xlsx')

# zakres dat
DATE_START=pd.to_datetime('2022-09-30')
DATE_R=DATE_START.year
DATE_END=datetime.datetime(DATE_R+5,12,31)

# parmetry finansowe
FX_EUR=4.7
DISCOUNT_RATE=9.0

INDEX=7.12
MARGIN=2.5
WACC=10

# zakres czasowy modelu
okres=pd.date_range(start=DATE_START,freq='M',end=DATE_END)
t_okres=pd.DataFrame({'DATE':okres})
t_okres['R']=t_okres['DATE'].dt.year
t_okres['M']=t_okres['DATE'].dt.month
t_okres['I']=t_okres.index

t_model=t_okres

## Bilans - Aktywa

#### Planowane nakłady inwestycyjne

In [242]:
# Nakłady Inwestycyjne (zadania) - do późniejszej dzierżawy (leasingu operacyjnego)

# A - inwestycje 'must-have'
# B - inwestycje wpływające na efektywność / wydajność
# C - inwestycje dodatkowe (przejęcie usług)

l_ni=[

# ----------------------------------------------------------------------
        {'Typ':                    'A',
        'Nazwa':                   'Ładowarka kołowa WA475-10',
        'Cena [PLN]':               1265000,
        'Data OT':                  '2022-11-30',
        'Amortyzacja [okres]':      7*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':   0},
# ---------------------------------------------------------------------
        {'Typ':                    'A',
        'Nazwa':                   'Węzeł uszlachetniania kruszyw (przesiewacza dwupokładowy)',
        'Cena [PLN]':               200000,
        'Data OT':                  '2022-12-31',
        'Amortyzacja [okres]':      6*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':   0},
# ----------------------------------------------------------------------
        {'Typ':                    'A',
        'Nazwa':                   'Węzeł uszlachetniania kruszyw (głowica zsypowa i adaptacja konstrukcji)',
        'Cena [PLN]':               100000,
        'Data OT':                  '2022-12-31',
        'Amortyzacja [okres]':      6*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':   0},
# ----------------------------------------------------------------------
        {'Typ':                    'A',
        'Nazwa':                   'Węzeł uszlachetniania kruszyw (wanna zrzutowa błota)',
        'Cena [PLN]':               100000,
        'Data OT':                  '2022-12-31',
        'Amortyzacja [okres]':      6*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':  0},
# ----------------------------------------------------------------------
        {'Typ':                    'A',
        'Nazwa':                   'Węzeł uszlachetniania kruszyw (adaptacja przenośnika na zkrężny)',
        'Cena [PLN]':               50000,
        'Data OT':                  '2022-12-31',
        'Amortyzacja [okres]':      6*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':   0},
# ----------------------------------------------------------------------
        {'Typ':                    'A',
        'Nazwa':                   'Węzeł uszlachetniania kruszyw (prace instalacyjne)',
        'Cena [PLN]':               50000,
        'Data OT':                  '2022-12-31',
        'Amortyzacja [okres]':      6*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':  0},
# ----------------------------------------------------------------------
        {'Typ':                    'A',
        'Nazwa':                   'Węzeł uszlachetniania kruszyw (sterówka)',
        'Cena [PLN]':               25000,
        'Data OT':                  '2022-12-31',
        'Amortyzacja [okres]':      6*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':  0},
# ----------------------------------------------------------------------
        {'Typ':                    'B',
        'Nazwa':                   'Kruszarka CH830 (RAT)',
        'Cena [PLN]':               290000*FX_EUR,
        'Data OT':                  '2023-03-31',
        'Amortyzacja [okres]':      10*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':  0},
# ----------------------------------------------------------------------
        {'Typ':                    'B',
        'Nazwa':                   'Kruszarka CH830 (Instalacja)',
        'Cena [PLN]':               60000*FX_EUR,
        'Data OT':                  '2023-03-31',
        'Amortyzacja [okres]':      10*12,
        'Udział własny':        100,
        'Okres finansowania': 0,
        'Wartość rezydualna':  0},
# ----------------------------------------------------------------------

] 

t_ni=pd.DataFrame(l_ni)

In [243]:


t_tabela=t_ni.copy()
t_tabela=t_tabela[['Nazwa', 'Cena [PLN]', 'Data OT','Udział własny','Okres finansowania','Wartość rezydualna']]
t_tabela.loc['RAZEM','Cena [PLN]']=t_tabela['Cena [PLN]'].sum()
t_tabela.loc['RAZEM','Nazwa']='RAZEM'
t_tabela_fv=t_tabela.style.format({'Cena [PLN]':'{0:,.2f}','Udział własny':'{0:,.2f}','Okres finansowania':'{0:,.2f}','Wartość rezydualna':'{0:,.2f}'}).hide(axis='index')
t_tabela_fv.to_latex(ASSETS / '1_Nakłady_Inwestycyjne.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}R{2cm}R{2cm}R{1.5cm}')
t_tabela_fv


Nazwa,Cena [PLN],Data OT,Udział własny,Okres finansowania,Wartość rezydualna
Ładowarka kołowa WA475-10,"1,265,000.00",2022-11-30,100.00,0.00,0.00
Węzeł uszlachetniania kruszyw (przesiewacza dwupokładowy),"200,000.00",2022-12-31,100.00,0.00,0.00
Węzeł uszlachetniania kruszyw (głowica zsypowa i adaptacja konstrukcji),"100,000.00",2022-12-31,100.00,0.00,0.00
Węzeł uszlachetniania kruszyw (wanna zrzutowa błota),"100,000.00",2022-12-31,100.00,0.00,0.00
Węzeł uszlachetniania kruszyw (adaptacja przenośnika na zkrężny),"50,000.00",2022-12-31,100.00,0.00,0.00
Węzeł uszlachetniania kruszyw (prace instalacyjne),"50,000.00",2022-12-31,100.00,0.00,0.00
Węzeł uszlachetniania kruszyw (sterówka),"25,000.00",2022-12-31,100.00,0.00,0.00
Kruszarka CH830 (RAT),"1,363,000.00",2023-03-31,100.00,0.00,0.00
Kruszarka CH830 (Instalacja),"282,000.00",2023-03-31,100.00,0.00,0.00
RAZEM,"3,435,000.00",nan,nan,nan,nan


#### Rozliczenie nakładów inwestycyjnych

In [244]:
# Rozliczenie inwestycji

t_capex=pd.DataFrame()

for index,row in t_ni.iterrows():

    NAME=row['Nazwa']
    CENA=row['Cena [PLN]']
    DATA_OT=pd.to_datetime(row['Data OT'])
    AM=row['Amortyzacja [okres]']
    DP=row['Udział własny']
    T=row['Okres finansowania']
    RV=row['Wartość rezydualna']

    okres=pd.date_range(start=DATA_OT,freq='M',periods=AM+1)
    t_capex_temp=pd.DataFrame({'DATE':okres})
    t_capex_temp['R']=t_capex_temp['DATE'].dt.year
    t_capex_temp['M']=t_capex_temp['DATE'].dt.month
    t_capex_temp['I']=t_capex_temp.index
    t_capex_temp['Nazwa']=NAME
    t_capex_temp['Zakup']=t_capex_temp.apply(lambda x:0 if x['DATE']!=DATA_OT else CENA, axis=1)
    t_capex_temp['Amortyzacja']=t_capex_temp.apply(lambda x:0 if x['I']==0 else CENA/AM, axis=1)
    t_capex_temp['Aktywo']=t_capex_temp['Zakup'].cumsum()-t_capex_temp['Amortyzacja'].cumsum()
    t_capex_temp['Finansowanie własne']=t_capex_temp.apply(lambda x:CENA*DP/100 if x['I']==0 else 0, axis=1)
    t_capex_temp['Finansowanie obce']=t_capex_temp.apply(lambda x:CENA*(100-DP)/100 if x['I']==0 else 0, axis=1)
    t_capex_temp['PPMT']=t_capex_temp.apply(lambda x:0 if (x['I']==0 or x['I']>T) else -npf.ppmt((INDEX+MARGIN)/1200,x['I'],T,CENA*(100-DP)/100,-CENA*(RV)/100),axis=1)
    t_capex_temp['IPMT']=t_capex_temp.apply(lambda x:0 if (x['I']==0 or x['I']>T) else -npf.ipmt((INDEX+MARGIN)/1200,x['I'],T,CENA*(100-DP)/100,-CENA*(RV)/100),axis=1)

    t_capex_temp['PMT']=t_capex_temp.apply(lambda x:0 if (x['I']==0 or x['I']>AM) else -npf.pmt((WACC)/1200,AM,CENA,0),axis=1)
    
    t_capex_temp['RV']=t_capex_temp.apply(lambda x:CENA*(RV)/100 if x['DATE']==DATA_OT+pd.DateOffset(months=T) else 0, axis=1)
    t_capex_temp['WNA']=t_capex_temp.apply(lambda x:0 if x['DATE']!=DATA_OT+pd.DateOffset(months=T) else x['Aktywo'], axis=1)

    t_capex=pd.concat([t_capex,t_capex_temp],ignore_index=True)

t_capex_model=t_capex.groupby(['DATE','R'],as_index=False)[['Zakup', 'Amortyzacja','Finansowanie własne', 'Finansowanie obce', 'PPMT', 'IPMT', 'RV','PMT']].sum()

temp=t_capex_model[['R','Zakup', 'Amortyzacja','Finansowanie własne', 'Finansowanie obce', 'PPMT', 'IPMT', 'RV','PMT']]
temp=temp.groupby(['R'],as_index=False)[['Zakup', 'Amortyzacja','Finansowanie własne', 'Finansowanie obce', 'PPMT', 'IPMT', 'RV','PMT']].sum()
temp.rename(columns={'Zakup':'Nakłady inwestycyjne','PPMT':'Spłata kapitału','IPMT':'Spłata odsetek','RV':'Wykup','PMT':'Przychody z wynajmu'},inplace=True)
temp['Zaangażowanie leasingu']=temp['Nakłady inwestycyjne']-temp['Finansowanie własne']-temp['Spłata kapitału']-temp['Wykup']
temp['Zaangażowanie leasingu']=temp['Zaangażowanie leasingu'].cumsum()
temp['Środek trwały']=temp['Nakłady inwestycyjne']-temp['Amortyzacja']
temp['Środek trwały']=temp['Środek trwały'].cumsum()

temp=temp[temp['R']<=DATE_R+5]


temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '2_Rozliczenie_Inwestycji.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Nakłady inwestycyjne,"1,790.00","1,645.00",0.00,0.00,0.00,0.00
Amortyzacja,15.06,391.59,432.71,432.71,432.71,432.71
Finansowanie własne,"1,790.00","1,645.00",0.00,0.00,0.00,0.00
Finansowanie obce,0.00,0.00,0.00,0.00,0.00,0.00
Spłata kapitału,0.00,0.00,0.00,0.00,0.00,0.00
Spłata odsetek,0.00,0.00,0.00,0.00,0.00,0.00
Wykup,0.00,0.00,0.00,0.00,0.00,0.00
Przychody z wynajmu,21.00,564.37,629.58,629.58,629.58,629.58
Zaangażowanie leasingu,0.00,0.00,0.00,0.00,0.00,0.00
Środek trwały,"1,774.94","3,028.35","2,595.64","2,162.92","1,730.21","1,297.49"


In [245]:
# Nakłady Inwestycyjne finansowane z obligacji

# A - inwestycje 'must-have'
# B - inwestycje wpływające na efektywność / wydajność
# C - inwestycje dodatkowe (przejęcie usług)

l_bond=[

# --------------------------------------------------------------------------------------
        {'Typ':                    'B',
        'Nazwa':                   'Węzeł wstępnego odsiewu (przemieszczenie nadkładu)',
        'Cena [PLN]':               420000
        },
# --------------------------------------------------------------------------------------
        {'Typ':                    'B',
        'Nazwa':                   'Węzeł wstępnego odsiewu (modernizacja)',
        'Cena [PLN]':               280000
        },
# --------------------------------------------------------------------------------------
        {'Typ':                    'B',
        'Nazwa':                   'Odkrywka północna XIa',
        'Cena [PLN]':               800000
        },
# --------------------------------------------------------------------------------------
        {'Typ':                    'B',
        'Nazwa':                   'Remonty i modernizacje',
        'Cena [PLN]':               597000
        },
# --------------------------------------------------------------------------------------

] 

t_bond=pd.DataFrame(l_bond)

kwota_obligacji=4900000 - t_ni['Cena [PLN]'].sum()
udział_libertas=kwota_obligacji/t_bond['Cena [PLN]'].sum()

t_bond['Udział (LIB)']=udział_libertas*100
t_bond['Obligacje [PLN]']=t_bond['Cena [PLN]']*udział_libertas


In [246]:
t_tabela=t_bond.copy()
t_tabela=t_tabela[['Nazwa', 'Cena [PLN]','Udział (LIB)','Obligacje [PLN]']]
#t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM']=t_tabela[['Cena [PLN]','Obligacje [PLN]']].sum(axis=0)
t_tabela.loc['RAZEM','Nazwa']='RAZEM'
t_tabela_fv=t_tabela.style.format({'Cena [PLN]':'{0:,.2f}','Udział (LIB)':'{0:,.2f}','Obligacje [PLN]':'{0:,.2f}'}).hide(axis='index')
t_tabela_fv.to_latex(ASSETS / '3_Nakłady_Inwestycyjne(obligacje).tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}R{2cm}')
t_tabela_fv

Nazwa,Cena [PLN],Udział (LIB),Obligacje [PLN]
Węzeł wstępnego odsiewu (przemieszczenie nadkładu),"420,000.00",69.86,"293,419.17"
Węzeł wstępnego odsiewu (modernizacja),"280,000.00",69.86,"195,612.78"
Odkrywka północna XIa,"800,000.00",69.86,"558,893.66"
Remonty i modernizacje,"597,000.00",69.86,"417,074.39"
RAZEM,"2,097,000.00",nan,"1,465,000.00"


In [247]:
t_tabela=t_remonty.copy()
t_tabela.fillna(0,inplace=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
#t_tabela.loc['RAZEM']=t_tabela[['Cena [PLN]','Obligacje [PLN]']].sum(axis=0)
t_tabela.loc['RAZEM','Obszar']='RAZEM'
t_tabela_fv=t_tabela.style.format({'Materiał':'{0:,.2f}','Usługa':'{0:,.2f}','Razem':'{0:,.2f}'}).hide(axis='index')
t_tabela_fv.to_latex(ASSETS / '4_Remonty.tex',hrules=True,column_format='L{4cm}L{5cm}R{2cm}R{2cm}R{2cm}')
t_tabela_fv

# Uzupełnić tex:
#\begin{longtable}{L{4cm}L{5cm}R{2cm}R{2cm}R{2cm}}
#\toprule
#Obszar & Zadanie & Materiał & Usługa & Razem \\
#\midrule
#\endhead

Obszar,Zadanie,Materiał,Usługa,Razem
Bezpieczeństwo pracy w koplani,Sól drogowa na sezon zimowy 30 ton,"28,500.00",0.00,"28,500.00"
Elektryka,Modernizacja rozdzielni Sn w zakresie kompesacji energii biernej - wyższa harmoniczna,0.00,"8,000.00","8,000.00"
Elektryka,Naprawa elektrozawrów systemu zraszania,"5,000.00",0.00,"5,000.00"
Elektryka,"Remont falowników 37KW, 30kW",0.00,"3,000.00","3,000.00"
Elektryka,Wymiana lamp na energooszczędne,"2,500.00",0.00,"2,500.00"
Elektryka,Stacja TRAFO 670kVa,"40,000.00","15,000.00","55,000.00"
Lewa strona - zakład kruszenia,Remont głowicy zsypowej - wymiana wykladek trudnościeralnych przesiewacz P109,"18,000.00",0.00,"18,000.00"
Lewa strona - zakład kruszenia,Remont głowicy zsypowej - wymiana wykladek trudnościeralnych przesiewacz P112,"18,000.00",0.00,"18,000.00"
Lewa strona - zakład kruszenia,Spawanie przesiewacza P112 + wzmocnienie elementów,"5,000.00",0.00,"5,000.00"
Lewa strona - zakład kruszenia,Wymiana kosza zasypowego nad kruszarkami stożkowymi CH,0.00,"3,000.00","3,000.00"


#### Aktualizacja modelu

In [248]:
# Aktualizacja modelu o nakłady inwestycyjne i ich finansowanie

t_capex_model['Aktywo']=t_capex_model['Zakup']-t_capex_model['Amortyzacja']
t_capex_model['Pasywo']=t_capex_model['Finansowanie obce']-t_capex_model['PPMT']-t_capex_model['RV']

temp=t_capex_model[['DATE']].copy()
temp['Nakłady inwestycyjne']=-t_capex_model['Zakup']
temp['Kredyty bankowe i leasingi (inwestycje)']=t_capex_model['Finansowanie obce']
temp['Finansowanie własne']=t_capex_model['Finansowanie własne']
temp['Amortyzacja (inwestycje)']=-t_capex_model['Amortyzacja']
temp['Koszty finansowe (inwestycje)']=-t_capex_model['IPMT']
temp['Spłata kredytów i leasingów (inwestycje)']=-t_capex_model['PPMT']-t_capex_model['RV']
temp['Rzeczowe aktywa trwałe (inwestycje)']=t_capex_model['Aktywo']
temp['Zobowiązania długoterminowe (inwestycje)']=t_capex_model['Pasywo']
t_model=pd.merge(t_model,temp,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)

In [249]:
# Raport

temp=t_model[['R','Nakłady inwestycyjne','Rzeczowe aktywa trwałe (inwestycje)','Amortyzacja (inwestycje)','Kredyty bankowe i leasingi (inwestycje)','Finansowanie własne','Zobowiązania długoterminowe (inwestycje)','Spłata kredytów i leasingów (inwestycje)','Koszty finansowe (inwestycje)']].copy()
temp=temp.groupby(['R'],as_index=False)[['Nakłady inwestycyjne','Rzeczowe aktywa trwałe (inwestycje)','Amortyzacja (inwestycje)','Kredyty bankowe i leasingi (inwestycje)','Finansowanie własne','Zobowiązania długoterminowe (inwestycje)','Spłata kredytów i leasingów (inwestycje)','Koszty finansowe (inwestycje)']].sum()

temp['Rzeczowe aktywa trwałe (inwestycje)']=temp['Rzeczowe aktywa trwałe (inwestycje)'].cumsum()
temp['Zobowiązania długoterminowe (inwestycje)']=temp['Zobowiązania długoterminowe (inwestycje)'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '5_Wpływ_Inwestycji_SF.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Nakłady inwestycyjne,"-1,790.00","-1,645.00",0.00,0.00,0.00,0.00
Rzeczowe aktywa trwałe (inwestycje),"1,774.94","3,028.35","2,595.64","2,162.92","1,730.21","1,297.49"
Amortyzacja (inwestycje),-15.06,-391.59,-432.71,-432.71,-432.71,-432.71
Kredyty bankowe i leasingi (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00
Finansowanie własne,"1,790.00","1,645.00",0.00,0.00,0.00,0.00
Zobowiązania długoterminowe (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00
Spłata kredytów i leasingów (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00
Koszty finansowe (inwestycje),0.00,0.00,0.00,0.00,0.00,0.00


## Rachunek Wyników

### Przychody

In [250]:
# Przychody z wynajmu

temp=t_capex_model[['DATE','PMT']].copy()
t_model=pd.merge(t_model,temp,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)

In [251]:
# Przychody z tytułu obligacji

BOND_VALUE=t_bond['Obligacje [PLN]'].sum()
WIBOR3M=7.61
BOND_MARGIN=2.0
BOND_EMISJA=pd.to_datetime('2022-12-31')

BOND_M_INCOME=BOND_VALUE*(WIBOR3M+BOND_MARGIN)/1200

t_model['Obligacje_Przychód']=t_model.apply(lambda x:BOND_M_INCOME if x['DATE']>BOND_EMISJA else 0,axis=1)
t_model['Obligacje_Emisja']=t_model.apply(lambda x:BOND_VALUE if x['DATE']==BOND_EMISJA else 0,axis=1)

### Koszty operacyjne

In [252]:
#'2. Zużycie materiałów i energii'

#'3. Usługi obce'

t_model['Opex_Księgowość']=-3000
t_model['Opex_Czynsz']=-172.82
t_model['Opex_Prawna']=t_model.apply(lambda x:-5000 if x['M']==12 else 0,axis=1)
t_model['Opex_RODO']=t_model.apply(lambda x:-3000 if x['M']==12 else 0,axis=1)

#'4. Podatki i opłaty' 
#'5. Wynagrodzenia'

t_model['Opex_Zarząd']=-12000

#'6. Ubezpieczenia społeczne' 

#'7. Pozostałe koszty rodzajowe'
t_model['Opex_Inne']=-1500

## Mapowanie sprawozdań

In [253]:
# Bilans otwarcia

t_model['Kapitał_BO']=t_model.apply(lambda x:5000000 if x['DATE']==DATE_START else 0,axis=1)
t_model['CF_BO']=t_model.apply(lambda x:5000000 if x['DATE']==DATE_START else 0,axis=1)

In [254]:
# Mapowanie rachunku wyników

# Przychody ze sprzedaży

t_model['A. PRZYCHODY NETTO']=t_model[['PMT']].sum(axis=1)

# Koszty operacyjne

t_model['1. Amortyzacja']=t_model[['Amortyzacja (inwestycje)']].sum(axis=1)
t_model['2. Zużycie materiałów i energii']=0
t_model['3. Usługi obce']=t_model[['Opex_Księgowość', 'Opex_Czynsz', 'Opex_Prawna', 'Opex_RODO']].sum(axis=1)
t_model['4. Podatki i opłaty']=0
t_model['5. Wynagrodzenia']=t_model[['Opex_Zarząd']].sum(axis=1)
t_model['6. Ubezpieczenia społeczne']=0
t_model['7. Pozostałe koszty rodzajowe']=t_model[['Opex_Inne']].sum(axis=1)
t_model['8. Wartość sprzedanych towarów i materiałów']=0

t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']=t_model[[ 
    '1. Amortyzacja',
    '2. Zużycie materiałów i energii', 
    '3. Usługi obce',
    '4. Podatki i opłaty', 
    '5. Wynagrodzenia',
    '6. Ubezpieczenia społeczne', 
    '7. Pozostałe koszty rodzajowe',
    '8. Wartość sprzedanych towarów i materiałów']].sum(axis=1)

# Pozostałe przychody i koszty

t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']=0
t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']=0

# Przychody i koszty finnasowe

t_model['E. PRZYCHODY FINANSOWE']=t_model[['Obligacje_Przychód']].sum(axis=1)
t_model['F. KOSZTY FINANSOWE']=t_model[['Koszty finansowe (inwestycje)']].sum(axis=1)


t_model['G. WYNIK BRUTTO']=t_model[[
    'A. PRZYCHODY NETTO',
    'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
    'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
    'D. POZOSTAŁE KOSZTY OPERACYJNE',
    'E. PRZYCHODY FINANSOWE',
    'F. KOSZTY FINANSOWE']].sum(axis=1)

t_model['TAX']=-t_model.apply(lambda x:x['G. WYNIK BRUTTO']*0.19 if x['G. WYNIK BRUTTO']>0 else 0,axis=1)

t_model['H. PODATEK DOCHODOWY']=t_model[['TAX']].sum(axis=1)

t_model['I. WYNIK NETTO']=t_model[[
    'G. WYNIK BRUTTO',
    'H. PODATEK DOCHODOWY']].sum(axis=1)

In [255]:
# Zapotrzebowanie na kapitał obrotowy

ROTACJA_NALEŻNOŚCI=0
ROTACJA_ZAPASÓW=0
ROTACJA_ZOBOWIĄZAŃ=0

t_model['1. Należności']=(t_model['A. PRZYCHODY NETTO']*ROTACJA_NALEŻNOŚCI)/365
t_model['2. Zapasy']=(t_model['A. PRZYCHODY NETTO']*ROTACJA_ZAPASÓW)/365
t_model['A. AKTYWA BIEŻĄCE']=t_model['1. Należności']+t_model['2. Zapasy']
t_model['1. Zobowiązania']=(-(t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']+t_model['1. Amortyzacja'])*ROTACJA_ZOBOWIĄZAŃ)/365
t_model['B. PASYWA BIEŻĄCE']=t_model['1. Zobowiązania']
t_model['C. KAPITAŁ OBROTOWY NETTO']=t_model['A. AKTYWA BIEŻĄCE']-t_model['B. PASYWA BIEŻĄCE']


In [256]:
# Przepływy pieniężne

#-----------------------------------
# WPŁYWY
#-----------------------------------

t_model['1. Kapitał zakładowy']=0
t_model['2. Kredyty bankowe']=t_model['Kredyty bankowe i leasingi (inwestycje)']
t_model['3. Leasing finansowy']=0
t_model['4. Inne']=0
t_model['5. Przychody ze sprzedaży']=t_model['A. PRZYCHODY NETTO']+t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']+t_model['E. PRZYCHODY FINANSOWE']

t_model['A. WPŁYWY PIENIĘŻNE OGÓŁEM']=t_model[['1. Kapitał zakładowy', '2. Kredyty bankowe', '3. Leasing finansowy','4. Inne', '5. Przychody ze sprzedaży']].sum(axis=1)

#-----------------------------------
# WYDATKI
#-----------------------------------

t_model['1. Nakłady inwestycyjne']=t_model['Nakłady inwestycyjne']
t_model['2. Koszty operacyjne (bez amortyzacji)']=t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']-t_model['1. Amortyzacja']
t_model['3. Pozostałe koszty operacyjne']=t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']
t_model['4. Spłaty kredytów i leasingów']=t_model['Spłata kredytów i leasingów (inwestycje)']
t_model['5. Odsetki']=t_model['F. KOSZTY FINANSOWE']
t_model['6. Podatek dochodowy']=t_model['H. PODATEK DOCHODOWY']
t_model['7. Objęcie obligacji']=-t_model['Obligacje_Emisja']

t_model['B. WYDATKI PIENIĘŻNE OGÓŁEM']=t_model[['1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)','3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów','5. Odsetki', '6. Podatek dochodowy','7. Objęcie obligacji']].sum(axis=1)

t_model['C. ZAPOTRZEBOWANIE NA KON']=t_model['C. KAPITAŁ OBROTOWY NETTO']

t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']=t_model['A. WPŁYWY PIENIĘŻNE OGÓŁEM']+t_model['B. WYDATKI PIENIĘŻNE OGÓŁEM']+t_model['C. ZAPOTRZEBOWANIE NA KON']
t_model['E. SKUMULOWANE SALDO ŚP']=t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']

In [257]:
# Mapowanie bilansu - Aktywa

t_model['1. Wartości niematerialne i prawne']=0
t_model['2. Rzeczowe aktywa trwałe']=t_model[['Rzeczowe aktywa trwałe (inwestycje)']].sum(axis=1)
t_model['3. Należności długoterminowe']=t_model['Obligacje_Emisja']
t_model['4. Długoterminowe rozliczenia międzyokresowe']=0

t_model['A. AKTYWA TRWAŁE']=t_model[[ 
    '1. Wartości niematerialne i prawne',
    '2. Rzeczowe aktywa trwałe',
    '3. Należności długoterminowe',
    '4. Długoterminowe rozliczenia międzyokresowe']].sum(axis=1)

t_model['1. Zapasy']=t_model[['2. Zapasy']].sum(axis=1)
t_model['2. Należności krótkoterminowe']=t_model[['1. Należności']].sum(axis=1)
t_model['3. Inwestycje krótkoterminowe']=t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']+t_model['CF_BO']
t_model['4. Krótkoterminowe rozliczenia międzyokresowe']=0.0

t_model['B. AKTYWA OBROTOWE']=t_model[[ 
    '1. Zapasy',
    '2. Należności krótkoterminowe',
    '3. Inwestycje krótkoterminowe',
    '4. Krótkoterminowe rozliczenia międzyokresowe']].sum(axis=1)

t_model['AKTYWA RAZEM']=t_model[['A. AKTYWA TRWAŁE','B. AKTYWA OBROTOWE']].sum(axis=1)



In [258]:
# Mapowanie bilansu - Pasywa

t_model['1. Kapitał podstawowy']=t_model['Kapitał_BO']
t_model['2. Kapitał zapasowy']=0
t_model['3. Kapitał z aktualizacji wyceny']=0
t_model['4. Pozostałe kapitały rezerwowe']=0
t_model['5. Zysk (strata) z lat ubiegłych']=0
t_model['6. Zysk (strata)']=t_model['I. WYNIK NETTO']
t_model['7. Odpisy w ciągu roku']=0

t_model['A. KAPITAŁ WŁASNY']=t_model[[ 
    '1. Kapitał podstawowy',
    '2. Kapitał zapasowy',
    '3. Kapitał z aktualizacji wyceny',
    '4. Pozostałe kapitały rezerwowe',
    '5. Zysk (strata) z lat ubiegłych',
    '6. Zysk (strata)',
    '7. Odpisy w ciągu roku']].sum(axis=1)

t_model['1. Rezerwy na zobowiązania']=0
t_model['2. Zobowiązania długoterminowe']=t_model[['Zobowiązania długoterminowe (inwestycje)']].sum(axis=1)
t_model['3. Zobowiązania krótkoterminowe']=-t_model['1. Zobowiązania']
t_model['4. Rozliczenia międzyokresowe']=0
t_model['5. Fundusze specjalne']=0

t_model['B. ZOBOWIĄZANIA I REZERWY']=t_model[[ 
    '1. Rezerwy na zobowiązania',
    '2. Zobowiązania długoterminowe',
    '3. Zobowiązania krótkoterminowe',
    '4. Rozliczenia międzyokresowe',
    '5. Fundusze specjalne']].sum(axis=1)

t_model['PASYWA RAZEM']=t_model[['A. KAPITAŁ WŁASNY', 'B. ZOBOWIĄZANIA I REZERWY']].sum(axis=1)

### RAPORT ROCZNY

In [259]:
# Raport roczny
temp=t_model[[
       'R',
       'A. PRZYCHODY NETTO',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
       '1. Amortyzacja',
       '2. Zużycie materiałów i energii',
       '3. Usługi obce',
       '4. Podatki i opłaty',
       '5. Wynagrodzenia',
       '6. Ubezpieczenia społeczne',
       '7. Pozostałe koszty rodzajowe',
       '8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
       'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE',
       'F. KOSZTY FINANSOWE',
       'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. PRZYCHODY NETTO',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
       '1. Amortyzacja',
       '2. Zużycie materiałów i energii',
       '3. Usługi obce',
       '4. Podatki i opłaty',
       '5. Wynagrodzenia',
       '6. Ubezpieczenia społeczne',
       '7. Pozostałe koszty rodzajowe',
       '8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
       'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE',
       'F. KOSZTY FINANSOWE',
       'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO']].sum()


temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '6_PL.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. PRZYCHODY NETTO,21.00,564.37,629.58,629.58,629.58,629.58
B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ,-89.75,-599.66,-640.79,-640.79,-640.79,-640.79
1. Amortyzacja,-15.06,-391.59,-432.71,-432.71,-432.71,-432.71
2. Zużycie materiałów i energii,0.00,0.00,0.00,0.00,0.00,0.00
3. Usługi obce,-20.69,-46.07,-46.07,-46.07,-46.07,-46.07
4. Podatki i opłaty,0.00,0.00,0.00,0.00,0.00,0.00
5. Wynagrodzenia,-48.00,-144.00,-144.00,-144.00,-144.00,-144.00
6. Ubezpieczenia społeczne,0.00,0.00,0.00,0.00,0.00,0.00
7. Pozostałe koszty rodzajowe,-6.00,-18.00,-18.00,-18.00,-18.00,-18.00
8. Wartość sprzedanych towarów i materiałów,0.00,0.00,0.00,0.00,0.00,0.00


In [260]:
# Zapotrzebowanie ma KO Tabela

temp=t_model[['R','A. AKTYWA BIEŻĄCE', '1. Należności', '2. Zapasy', 'B. PASYWA BIEŻĄCE', '1. Zobowiązania']].copy()
temp=temp.groupby(['R'],as_index=False)[['A. AKTYWA BIEŻĄCE', '1. Należności', '2. Zapasy', 'B. PASYWA BIEŻĄCE', '1. Zobowiązania']].sum()
temp['C. KAPITAŁ OBROTOWY NETTO']=temp['A. AKTYWA BIEŻĄCE']-temp['B. PASYWA BIEŻĄCE']
temp['temp']=temp.shift(1)['C. KAPITAŁ OBROTOWY NETTO']
temp['D. ZAPOTRZEBOWANIE NA KON']=-temp.apply(lambda x:x['C. KAPITAŁ OBROTOWY NETTO'] if x['R']==DATE_START.year else x['C. KAPITAŁ OBROTOWY NETTO']-x['temp'],axis=1)
temp.drop(['temp'],axis=1,inplace=True)

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '7_KON.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. AKTYWA BIEŻĄCE,0.00,0.00,0.00,0.00,0.00,0.00
1. Należności,0.00,0.00,0.00,0.00,0.00,0.00
2. Zapasy,0.00,0.00,0.00,0.00,0.00,0.00
B. PASYWA BIEŻĄCE,0.00,0.00,0.00,0.00,0.00,0.00
1. Zobowiązania,0.00,0.00,0.00,0.00,0.00,0.00
C. KAPITAŁ OBROTOWY NETTO,0.00,0.00,0.00,0.00,0.00,0.00
D. ZAPOTRZEBOWANIE NA KON,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00


In [261]:
# Cash Flow - Tabela


temp=t_model[['R','A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', '7. Objęcie obligacji','C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']]

temp=temp.groupby(['R'],as_index=False)[['A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', '7. Objęcie obligacji','C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']].sum()

temp['E. SKUMULOWANE SALDO ŚP']=temp['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH'].cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '8_CF.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. WPŁYWY PIENIĘŻNE OGÓŁEM,21.00,705.15,770.37,770.37,770.37,770.37
1. Kapitał zakładowy,0.00,0.00,0.00,0.00,0.00,0.00
2. Kredyty bankowe,0.00,0.00,0.00,0.00,0.00,0.00
3. Leasing finansowy,0.00,0.00,0.00,0.00,0.00,0.00
4. Inne,0.00,0.00,0.00,0.00,0.00,0.00
5. Przychody ze sprzedaży,21.00,705.15,770.37,770.37,770.37,770.37
B. WYDATKI PIENIĘŻNE OGÓŁEM,"-3,329.69","-1,873.12",-232.69,-232.69,-232.69,-232.69
1. Nakłady inwestycyjne,"-1,790.00","-1,645.00",0.00,0.00,0.00,0.00
2. Koszty operacyjne (bez amortyzacji),-74.69,-208.07,-208.07,-208.07,-208.07,-208.07
3. Pozostałe koszty operacyjne,0.00,0.00,0.00,0.00,0.00,0.00


In [262]:
# Raport roczny
temp=t_model[[
       'R',
       'A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].sum()

temp=temp.cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '9_Aktywa.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. AKTYWA TRWAŁE,"3,239.94","4,493.35","4,060.64","3,627.92","3,195.21","2,762.49"
1. Wartości niematerialne i prawne,0.00,0.00,0.00,0.00,0.00,0.00
2. Rzeczowe aktywa trwałe,"1,774.94","3,028.35","2,595.64","2,162.92","1,730.21","1,297.49"
3. Należności długoterminowe,"1,465.00","1,465.00","1,465.00","1,465.00","1,465.00","1,465.00"
4. Długoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00,0.00
B. AKTYWA OBROTOWE,"1,691.31",523.35,"1,061.02","1,598.70","2,136.38","2,674.05"
1. Zapasy,0.00,0.00,0.00,0.00,0.00,0.00
2. Należności krótkoterminowe,0.00,0.00,0.00,0.00,0.00,0.00
3. Inwestycje krótkoterminowe,"1,691.31",523.35,"1,061.02","1,598.70","2,136.38","2,674.05"
4. Krótkoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00,0.00


In [263]:
# Raport roczny
temp=t_model[[
       'R',
      'A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].sum()

temp=temp.cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')

temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5])
tabela=temp[['POZYCJA',DATE_R,DATE_R+1,DATE_R+2,DATE_R+3,DATE_R+4,DATE_R+5]].copy()
tabela=tabela.style.format({DATE_R:'{0:,.2f}',DATE_R+1:'{0:,.2f}',DATE_R+2:'{0:,.2f}',DATE_R+3:'{0:,.2f}',DATE_R+4:'{0:,.2f}',DATE_R+5:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '10_Pasywa.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. KAPITAŁ WŁASNY,"4,931.25","5,016.70","5,121.66","5,226.62","5,331.58","5,436.55"
1. Kapitał podstawowy,"5,000.00","5,000.00","5,000.00","5,000.00","5,000.00","5,000.00"
2. Kapitał zapasowy,0.00,0.00,0.00,0.00,0.00,0.00
3. Kapitał z aktualizacji wyceny,0.00,0.00,0.00,0.00,0.00,0.00
4. Pozostałe kapitały rezerwowe,0.00,0.00,0.00,0.00,0.00,0.00
5. Zysk (strata) z lat ubiegłych,0.00,0.00,0.00,0.00,0.00,0.00
6. Zysk (strata),-68.75,16.70,121.66,226.62,331.58,436.55
7. Odpisy w ciągu roku,0.00,0.00,0.00,0.00,0.00,0.00
B. ZOBOWIĄZANIA I REZERWY,0.00,0.00,0.00,0.00,0.00,0.00
1. Rezerwy na zobowiązania,0.00,0.00,0.00,0.00,0.00,0.00
